In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

model = BertForMaskedLM.from_pretrained('bert-large-uncased')
model.eval()
%%capture

UsageError: Line magic function `%%capture` not found.


In [3]:
def random_int_list(start, stop, length):

    import random

    start, stop = (int(start), int(stop)) if start <= stop else (int(stop), int(start))
    length = int(abs(length)) if length else 0
    random_list = []
    for i in range(length):
        random_list.append(random.randint(start, stop))
    return random_list

In [4]:
import pickle
with open(r'D:\Fordham\DS Research\submission\20201124\text_dict2.pickle', 'rb') as file:
    dic = pickle.load(file)

In [5]:
def predict3(text, correct_word_list, tokenizer, model):
    
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    masked_list = []
    for i in range(len(tokenized_text)):
        if tokenized_text[i] == "[MASK]":
            masked_list.append(i)
            
    segments_ids = [0] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
        
#     prediction_test = predictions.clone().detach()
    
    predict_list=[]
    for i in masked_list:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        predict_list.append(predicted_token)
        
    for i in range(1,len(predict_list)):
        if len(correct_word_list[i]) <= 2:
            continue
        else:
            let_loc = random_int_list(0,len(correct_word_list[i])-1,2)
            let_loc.sort()
            for j in range(len(predictions[0,masked_list[i]])):
                predicted_index = torch.argmax(predictions[0, masked_list[i]]).item()
                predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
                
                try:
                    if correct_word_list[i][let_loc[0]] != predicted_token[let_loc[0]] or correct_word_list[i][let_loc[1]] != predicted_token[let_loc[1]]:
                        predictions[0,masked_list[i],int(torch.argmax(predictions[0,masked_list[i]]))] = 0
                        continue
                    else:
                        predict_list[i] =  predicted_token
                        break
                except:
                    predictions[0,masked_list[i],int(torch.argmax(predictions[0,masked_list[i]]))] = 0
                    continue
    print("----------------------------------------------")
    print("Predicted word | Original word: ")
    for i in range(len(correct_word_list)):
        print(predict_list[i]," | ",correct_word_list[i])
    print("Paragraph: ")
    print(text)
    print("----------------------------------------------")
        
    return predict_list

In [6]:
def model_test3(text_dict,tokenizer, model):
    
    total_num = 0
    correct_num = 0
    for text, word in text_dict.items():
        total_num = total_num+len(word)
        try:
            predict_list = predict3(text,word,tokenizer,model)
        except:
            continue
        for i in range(len(word)):
                if word[i].lower() == predict_list[i].lower():
                    correct_num+=1
                
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    print("The number of total masked word: ",total_num)
    print("The number of predicted word correctly: ",correct_num)
    print("Accuracy rate: ",correct_num/total_num)
    print("-----------------------------------------------------------")
    print("-----------------------------------------------------------")
    
    return 

In [7]:
model_test3(dic,tokenizer,model)

----------------------------------------------
Predicted word | Original word: 
fans  |  fans
that  |  why
Paragraph: 
[CLS] I am sure some bashers of Pens [MASK] are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils . Actually , I am bit puzzled too and a bit relieved . However , I am going to put an end to non-PIttsburghers ' relief with a bit of praise for the Pens . Man , they are killing those Devils worse than I thought . Jagr just showed you [MASK] he is much better than his regular season stats . He is also a lot fo fun to watch in the playoffs . Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway . I was very disappointed not to see the Islanders lose the final regular season game . PENS RULE ! ! ! [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
is  |  is
?  |  on
Para

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1580 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
think  |  believe
most  |  most
Paragraph: 
[CLS] How about Kirlian imaging ? I [MASK] the FAQ for sci.skeptics ( sp ? ) has a nice write-up on this . They would certainly be [MASK] supportive on helping you to build such a device and connect to a 120Kvolt supply so that you can take a serious look at your `` aura '' ... : - ) [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
is  |  is
Paragraph: 
[CLS] There [MASK] no notion of heliocentric , or even galacticentric either . -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
read  |  find
suggest  |  suggest
Paragraph: 
[CLS] In the following report : _Turkey Eyes Regional Role

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (998 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
yes  |  Well
by  |  by
Paragraph: 
[CLS] ed > 1 . All of us that argue about gyroscopes , etc. , throughly understand ed > the technique of countersteering . me > Including all the ones who think that they countersteer all the way me > through a corner ? ? ed > [MASK] ... all the way through a decreasing-radius corner , anyway ... Maybe they are riding around an ever-decreasing circle of lies which eventually leads to the truth ... . me > The official line here ( though I do have my doubts about it ) is that the me > front brake is applied first , followed [MASK] the rear brake , the idea being me > that you avoid locking up the rear after weight transfer takes place . Me too , though unfortunately the `` Official Line '' is the one that you have to adhere to if you want to get a full licence . The examiner 's guidelines are laid down by the government , and the basic rider education courses have no choice 

...

----------------------------------------------
Predicted word | Original word: 
be  |  be
decides  |  decides
Paragraph: 
[CLS] I disagree with these . What society thinks should [MASK] irrelevant . What the individual [MASK] is all that is important . I think this is fairly obvious Generally by what they `` feel '' is right , which is the most idiotic policy I can think of . By thinking for ourselves . [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
is  |  is
date  |  date
Paragraph: 
[CLS] I know this [MASK] the wrong newsgroup , but I was wondering if anyone knows the [MASK] ( s ) of this year 's COMDEX convention in Atlanta , GA . Or has it passed already ? ? E-mail if possible . Thanks . [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
others  |  newbies
all  |  all
Paragraph: 
[CLS] Feeding frenzy ! ! ! Ah , 't is

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (841 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (534 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
/  |  binary
america  |  us
Paragraph: 
[CLS] Oops , I forgot to set read permission . It 's fixed now . ftp netcom.com login : anonymous password : your @ email.address cd pub/mvp [MASK] get clinton.zip You need pkzip 2.x or the latest net.zip to un- '' deflate '' this . Economic stats since Day One , plus all of the myriad ways Slick Willie and the Gang of 535 are preparing to do it to [MASK] . From Ron Brown 's desk , so any distortion is pro-Democrat , can you believe it ? [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
medical  |  malarial
rules  |  rules
Paragraph: 
[CLS] The FDA , I believe . Rules say no blood or blood products donations from anyone who has been in a [MASK] area for 3 years . I was a platelet donor until my Thailand trip and my blood bank was very disappointed to find out they could n't use me for 3

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (542 > 512). Running this sequence through BERT will result in indexing errors


----------------------------------------------
Predicted word | Original word: 
not  |  not
little  |  little
Paragraph: 
[CLS] james , i really hate to do this , but try reading the damn posts ! never was a t'bird mentioned . The discussion was about SHO 's and 'stangs [MASK] being up to spec . I do not know about t'birds . I only know that the specs quoted for the SHO by previous poster sounded a [MASK] anemic for me to say that it was up to snuff . This does not in any way disencourage* me from wishing to own one , nor does it make it a bad car . It merely means that i think Ford could have added that extra bit of safety and tossed in larger brakes , as the wheels are plenty large enough for them to fit ( if memory serves right , which it may very well not ) and the motor plenty powerful enough to need it . [SEP]
----------------------------------------------
----------------------------------------------
Predicted word | Original word: 
want  |  wanted
about  |  about
Paragraph: 
[